# Yahoo!ショッピングのカテゴリーＩＤ一覧を取得する

In [48]:
import requests
import json
import time
import csv
import pickle

# end point
url_cat = 'https://shopping.yahooapis.jp/ShoppingWebService/V1/json/categorySearch'
with open('yahooappid.txt', 'r') as f:
    appid = f.read()

all_categories_file = './all_categories.csv'

# APIリクエスト呼び出し用関数
def r_get(url, dct):
    time.sleep(1)
    return requests.get(url, params=dct)

# カテゴリ取得用関数
def get_cats(cat_id):
    try:
        result = r_get(url_cat, {'appid': appid, 'category_id': cat_id})
        cats = result.json()['ResultSet']['0']['Result']['Categories']['Children']
        #print(cats) if isinstance(cats, list) else print()
        for i, cat in cats.items():
            if not i == '_container':
                yield cat['Id'], {'short': cat['Title']['Short'],
                                  'medium': cat['Title']['Medium'], 
                                  'long': cat['Title']['Long']}
    except Exception as e:
        pass
        print(e)
    
def get_reviews(cat_id, max_items):
    items = 0
    results = []
    start = 1
    
    num_results = 50
    num_reviews_per_cat = 99999999
    
    while items < max_items:
        result = r_get(url_review, {'appid': appid, 'category_id': cat_id, 'results': num_results, 'start': start})
        if result.ok:
            rs = result.json()['ResultSet']
        else:
            print(f'error: [cat id] {cat_id} [reason] {result.status_code}-{result.reason}')
            if result.status_code == 400:
                print('ステータスコード400(badrequestは中止せず読み飛ばします)')
                break
            else:
                exit(True)
        
        avl = int(rs['totalResultsAvailable'])
        pos = int(rs['firstResultPosition'])
        ret = int(rs['totalResultsReturned'])
        
        print(f'総ヒット数：{avl}\t開始位置：{pos}\t取得数：{ret}')
        
        reviews = result.json()['ResultSet']['Result']
        for rev in reviews:
            desc_len = len(rev['Description'])
            if min_len < desc_len or max_len < desc_len:
                continue
            items += 1
            buff = {}
            buff['id'] = items
            buff['title'] = rev['ReviewTitle'].replace('\n', '').replace(',', '、')
            buff['rate'] = int(float(rev['Ratings']['Rate']))
            buff['comment'] = rev['Description'].replace('\n', '').replace(',', '、')
            buff['name'] = rev['Target']['Name']
            buff['code'] = rev['Target']['Code']
            results.append(buff)
            if items >= max_items:
                break
        start += ret
        print(f'有効件数：{items}')
    return results
        

# カテゴリー一覧ＣＳＶファイルの生成

In [35]:
output_buffer = [['カテゴリコードlv1', 'カテゴリコードlv2', 'カテゴリコードlv3',
                 'カテゴリ名lv1', 'カテゴリ名lv2', 'カテゴリ名lv3', 'カテゴリ名lv3_long']]

with open(all_categories_file, 'w') as f:
    writer = csv.writer(f, lineterminator='\n')
    writer.writerows(output_buffer)
    output_buffer = []

for id1, title1 in get_cats(1):
    print('カテゴリレベル１: ', title1['short'])
    try:
        for id2, title2 in get_cats(id1):
            for id3, title3 in get_cats(id2):
                wk = [id1, id2, id3, title1['short'], title2['short'], title3['short'], title3['long']]
                output_buffer.append(wk)
                with open(all_categories_file, 'a') as f:
                    writer = csv.writer(f, lineterminator='\n')
                    writer.writerows(output_buffer)
                    output_buffer = []
    except KeyError as e:
        print(e)
        continue
                

カテゴリレベル１:  ファッション
カテゴリレベル１:  食品
'list' object has no attribute 'items'
'list' object has no attribute 'items'
カテゴリレベル１:  アウトドア、釣り、旅行用品
'list' object has no attribute 'items'
カテゴリレベル１:  ダイエット、健康
'list' object has no attribute 'items'
'list' object has no attribute 'items'
'list' object has no attribute 'items'
'list' object has no attribute 'items'
'list' object has no attribute 'items'
'list' object has no attribute 'items'
カテゴリレベル１:  コスメ、美容、ヘアケア
'list' object has no attribute 'items'
'list' object has no attribute 'items'
カテゴリレベル１:  スマホ、タブレット、パソコン
'list' object has no attribute 'items'
'list' object has no attribute 'items'
カテゴリレベル１:  テレビ、オーディオ、カメラ
カテゴリレベル１:  家電
カテゴリレベル１:  家具、インテリア
'list' object has no attribute 'items'
カテゴリレベル１:  花、ガーデニング
'list' object has no attribute 'items'
'list' object has no attribute 'items'
'list' object has no attribute 'items'
'list' object has no attribute 'items'
'list' object has no attribute 'items'
'list' object has no attribute 'items'
'list' object h

In [36]:
import pandas as pd
from IPython.display import display

df = pd.read_csv(all_categories_file)
display(df.head())

,カテゴリコードlv1,カテゴリコードlv2,カテゴリコードlv3,カテゴリ名lv1,カテゴリ名lv2,カテゴリ名lv3,カテゴリ名lv3_long
0,13457,2494,37019,ファッション,レディースファッション,コート,レディースファッション > コート
1,13457,2494,37052,ファッション,レディースファッション,ジャケット,レディースファッション > ジャケット
2,13457,2494,36861,ファッション,レディースファッション,トップス,レディースファッション > トップス
3,13457,2494,36913,ファッション,レディースファッション,ボトムス,レディースファッション > ボトムス
4,13457,2494,36887,ファッション,レディースファッション,ワンピース、チュニック,レディースファッション > ワンピース、チュニック


In [49]:
url_review = 'https://shopping.yahooapis.jp/ShoppingWebService/V1/json/reviewSearch'

with open('yahooappid.txt', 'r') as f:
    appid = f.read()

max_len = 10000
min_len = 50

code = 49331
count = 5
result = get_reviews(code, count)
print(json.dumps(result, indent=2, ensure_ascii=False))


総ヒット数：9392	開始位置：1	取得数：50
有効件数：5
[
  {
    "id": 1,
    "title": "大容量バッテリーに偽りなし！素晴らしい…",
    "rate": 5,
    "comment": "大容量バッテリーに偽りなし！素晴らしい電池もちです。配送も丁寧で良かったです。",
    "name": "シャープ AQUOS sense 4 plus SH-M16(パープル) 8GB/128GB SIMフリー",
    "code": "best-denki_4974019169088"
  },
  {
    "id": 2,
    "title": "問題なく使用できた。こうにゅうしてせい…",
    "rate": 5,
    "comment": "問題なく使用できた。こうにゅうしてせいかいだった。",
    "name": "【大人気スマホ入門機】 UMIDIGI A7S SIMフリースマホ 本体 新品 |非接触体温計|6.53インチ|3カメラ|4150mAh|Andoroid10|2回線同時待受|技適あり",
    "code": "yesmobile_umidigi-a3s"
  },
  {
    "id": 3,
    "title": "とても早く届きました。楽しみにしていた…",
    "rate": 5,
    "comment": "とても早く届きました。楽しみにしていたので、嬉しかったです。梱包もしっかりされていて、安心しました。",
    "name": "新品未使用 SIMロック解除 SOV39 Xperia XZ3 ボルドーレッド スマホ 本体 白ロム",
    "code": "eco-sty_358544093412463"
  },
  {
    "id": 4,
    "title": "思っていたよりも早く届いて良かったです…",
    "rate": 5,
    "comment": "思っていたよりも早く届いて良かったです。商品もキズも無くキレイでした。",
    "name": "Aランク SIMフリー docomo SO-02J Xperia X Compact White SIMフリー 中古 スマホ 本体 あすつく対応  保証あり 白ロ